In [ ]:
import datetime
from ib_async import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

: 

In [24]:
ac.delete_library('pnl')

In [2]:
ib = connect_to_IB(clientid=2)

In [3]:
ac = initialize_db('db')
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [10]:
trade = Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=16, clientId=2, permId=1769050860, action='BUY', totalQuantity=1.0, lmtPrice=0.99, auxPrice=0.0, orderRef='AAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Patient')]), orderStatus=OrderStatus(orderId=16, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1769050860, parentId=0, lastFillPrice=0.0, clientId=2, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 6, 18, 24, 12, 56742, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 6, 18, 24, 12, 117234, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')

In [17]:
str(trade)

"Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=16, clientId=2, permId=1769050860, action='BUY', totalQuantity=1.0, lmtPrice=0.99, auxPrice=0.0, orderRef='AAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Patient')]), orderStatus=OrderStatus(orderId=16, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1769050860, parentId=0, lastFillPrice=0.0, clientId=2, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 6, 18, 24, 12, 56742, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 6, 18, 24, 12, 117234, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')"

In [14]:
trade.order.totalQuantity
trade.contract

Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS')

In [ ]:
strategy_symbol = 'test'
trade_dict = {'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
                'symbol': trade.contract.symbol,
                'asset class': trade.contract.secType,
                'position': trade.order.totalQuantity,
                '% of nav': 0, # to be calculated
                'averageCost': trade.orderStatus.avgFillPrice,
                'marketPrice': trade.orderStatus.lastFillPrice,
                'pnl %': 0, # to be calculated
                'strategy': trade.order.orderRef if trade.order.orderRef else strategy_symbol,
                'contract': trade.contract,
                'trade': str(trade),
                'open_dt':datetime.date.today().isoformat(),
                'close_dt': '',
                'deleted': False,
                'delete_dt': '',
                'marketValue': 0, # to be calculated
                'unrealizedPNL': 0, # to be calculated
                'currency':trade.contract.currency,
                'realizedPNL': 0, # to be calculated
                'account': self.account_id if self.account_id else ib.managedAccounts()[0],
                'marketValue_base': None,
                'fx_rate': None}

In [26]:
import pickle
serialized_trade = pickle.dumps(trade)

serialized_trade

b'\x80\x04\x95h\x10\x00\x00\x00\x00\x00\x00\x8c\x0fib_insync.order\x94\x8c\x05Trade\x94\x93\x94)\x81\x94}\x94(\x8c\x08contract\x94\x8c\x12ib_insync.contract\x94\x8c\x05Stock\x94\x93\x94)\x81\x94}\x94(\x8c\x07secType\x94\x8c\x03STK\x94\x8c\x05conId\x94J~\r\x04\x00\x8c\x06symbol\x94\x8c\x04AAPL\x94\x8c\x1clastTradeDateOrContractMonth\x94\x8c\x00\x94\x8c\x06strike\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x05right\x94h\x11\x8c\nmultiplier\x94h\x11\x8c\x08exchange\x94\x8c\x05SMART\x94\x8c\x0fprimaryExchange\x94\x8c\x06ISLAND\x94\x8c\x08currency\x94\x8c\x03USD\x94\x8c\x0blocalSymbol\x94h\x0f\x8c\x0ctradingClass\x94\x8c\x03NMS\x94\x8c\x0eincludeExpired\x94\x89\x8c\tsecIdType\x94h\x11\x8c\x05secId\x94h\x11\x8c\x0bdescription\x94h\x11\x8c\x08issuerId\x94h\x11\x8c\x10comboLegsDescrip\x94h\x11\x8c\tcomboLegs\x94]\x94\x8c\x14deltaNeutralContract\x94Nub\x8c\x05order\x94h\x00\x8c\nLimitOrder\x94\x93\x94)\x81\x94}\x94(\x8c\x07orderId\x94K\x10\x8c\x08clientId\x94K\x02\x8c\x06permId\x94J\xec\x92qi\x8c\

In [28]:
# Convert the serialized object to a string for storage
serialized_trade.decode('latin1')

'\x80\x04\x95h\x10\x00\x00\x00\x00\x00\x00\x8c\x0fib_insync.order\x94\x8c\x05Trade\x94\x93\x94)\x81\x94}\x94(\x8c\x08contract\x94\x8c\x12ib_insync.contract\x94\x8c\x05Stock\x94\x93\x94)\x81\x94}\x94(\x8c\x07secType\x94\x8c\x03STK\x94\x8c\x05conId\x94J~\r\x04\x00\x8c\x06symbol\x94\x8c\x04AAPL\x94\x8c\x1clastTradeDateOrContractMonth\x94\x8c\x00\x94\x8c\x06strike\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x05right\x94h\x11\x8c\nmultiplier\x94h\x11\x8c\x08exchange\x94\x8c\x05SMART\x94\x8c\x0fprimaryExchange\x94\x8c\x06ISLAND\x94\x8c\x08currency\x94\x8c\x03USD\x94\x8c\x0blocalSymbol\x94h\x0f\x8c\x0ctradingClass\x94\x8c\x03NMS\x94\x8c\x0eincludeExpired\x94\x89\x8c\tsecIdType\x94h\x11\x8c\x05secId\x94h\x11\x8c\x0bdescription\x94h\x11\x8c\x08issuerId\x94h\x11\x8c\x10comboLegsDescrip\x94h\x11\x8c\tcomboLegs\x94]\x94\x8c\x14deltaNeutralContract\x94Nub\x8c\x05order\x94h\x00\x8c\nLimitOrder\x94\x93\x94)\x81\x94}\x94(\x8c\x07orderId\x94K\x10\x8c\x08clientId\x94K\x02\x8c\x06permId\x94Jì\x92qi\x8c\x06a

In [31]:
str_trade = str(trade)
trade_recon = eval(str_trade)
trade_recon.contract

Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS')

In [6]:
strategy=''
if strategy:
    print(True)

In [8]:
account_id = ib.managedAccounts()[0]
lib = ac.get_library('portfolio')
lib.list_symbols()
lib.read('U7706434_SMAG7').data

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,TSLA,SMAG7,STK,2024-04-01 12:35,-25.0,-3.873397,176.650212,178.059998,-0.798066,"Stock(conId=76792991, symbol='TSLA', right='0'...",...,,False,,-4451.5,-35.24,USD,0.0,U7706434,-4130.101682,1.077819


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: . Keine Verbindung zu folgenden Datenzentren: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erh

In [26]:
pnl_lib = ac.get_library('pnl')


In [27]:
total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")
total_equity

105213.53

In [35]:
def get_total_equity(ib_client):
    """Fetches the total equity value from the account summary."""
    total_equity = sum(float(entry.value) for entry in ib_client.accountSummary() if entry.tag == "EquityWithLoanValue")
    return total_equity

def save_pnl(ac, total_equity, account_id):
    """Saves the PnL (equity value) to the ArcticDB."""
    pnl_lib = ac.get_library('pnl')
     # Truncate the current timestamp to an hourly granularity
    current_time = datetime.datetime.now().replace(second=0, microsecond=0)
    
    pnl_data = {
        'total_equity': total_equity,
        'account_id': account_id
    }
    pnl_df = pd.DataFrame([pnl_data], index=[current_time])
    try:
        # Append the new data to the 'pnl' library, creating it if it doesn't exist
        if 'pnl' in pnl_lib.list_symbols():
            pnl_lib.append(account_id, pnl_df)
        else:
            pnl_lib.write(account_id, pnl_df)
        print(f"Equity value saved to 'pnl' library for account {account_id}")
    except Exception as e:
        print(f"Error saving equity value to 'pnl' library: {e}")


In [36]:
account_id = ib.managedAccounts()[0]  # Assuming you want the first managed account's ID
total_equity = get_total_equity(ib)
save_pnl(ac, total_equity, account_id)

Equity value saved to 'pnl' library for account U7706434


In [37]:
pnl_data = pnl_lib.read(account_id).data
pnl_data

,total_equity,account_id
2024-03-26 15:00:00,105212.87,U7706434
